[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/blockworks-foundation%2Fmango-explorer-examples/HEAD?filepath=ShowOwnOrders.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Show Own Orders

This notebook shows how to retrieve only your own orders on a market using the `load_my_orders()` method.

This code will place two orders on SOL-PERP, pause for 10 seconds, then show all open orders, then just open orders from the current `Account`, then cancel those two orders.


In [ ]:
import decimal
import mango
import time

# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146])

# Create a 'devnet' Context
context = mango.ContextBuilder.build(cluster_name="devnet")

# Load the user's account
group = mango.Group.load(context)
accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
account = accounts[0]

# Load the market
stub = context.market_lookup.find_by_symbol("SOL-PERP")
market = mango.ensure_market_loaded(context, stub)

market_operations = mango.create_market_operations(context, wallet, account, market, dry_run=False)

print("Orders (initial):")
for order in market_operations.load_orders():
    print(order)

# Go on - try to buy 1 SOL for $10.
order1 = mango.Order.from_basic_info(side=mango.Side.BUY,
                                     price=decimal.Decimal(10),
                                     quantity=decimal.Decimal(1),
                                     order_type=mango.OrderType.POST_ONLY)
placed_order1 = market_operations.place_order(order1)

# And $11.
order2 = mango.Order.from_basic_info(side=mango.Side.BUY,
                                     price=decimal.Decimal(11),
                                     quantity=decimal.Decimal(1),
                                     order_type=mango.OrderType.POST_ONLY)
placed_order2 = market_operations.place_order(order2)
print("\n\nplaced_order\n\t", placed_order1, "\n\t", placed_order2)

print("\n\nSleeping for 10 seconds...")
time.sleep(10)

print("\n\nOrders (including our new orders):")
for order in market_operations.load_orders():
    print(order)

print("\n\nOur Own Orders:")
for order in market_operations.load_my_orders():
    print(order)

cancellaton_signatures1 = market_operations.cancel_order(placed_order1)
cancellaton_signatures2 = market_operations.cancel_order(placed_order2)
print("\n\ncancellaton_signatures:\n\t", cancellaton_signatures1 + cancellaton_signatures2)
